In [131]:
import openmc
import openmc.mgxs
import math
import matplotlib.pyplot as plt
import numpy as np
import pandas
import h5py
%matplotlib inline

In [132]:
import os

In [133]:
os.environ['PATH'] += r':/home/jae123p/miniconda3/envs/openmc-env/bin'

%env OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml

env: OPENMC_CROSS_SECTIONS=/home/jae123p/endfb80/endfb-viii.0-hdf5/cross_sections.xml


In [134]:
model = openmc.Model()

In [135]:
oralloy = openmc.Material(name="Oralloy")
oralloy.add_nuclide('U234', 4.9210e-04)
oralloy.add_nuclide('U235', 4.4917e-02)
oralloy.add_nuclide('U238', 2.5993e-03)

materials = openmc.Materials([oralloy])

In [119]:
model.materials = openmc.Materials([oralloy])

In [120]:
box = openmc.model.RectangularPrism(width=10, height = 10, boundary_type='reflective')
mat_cell = openmc.Cell(name='OralloyCell', fill = oralloy, region = -box)

In [121]:
root_universe = openmc.Universe()
root_universe.add_cell(mat_cell)
model.geometry = openmc.Geometry(root_universe)

In [122]:
bounds = [-10, -10, -10, 10, 10, 10]
uniform_dist = openmc.stats.Box(bounds[:3], bounds[3:])
energy_dist = openmc.stats.Uniform(a=0.0e6,b=20.0e6)
source = openmc.IndependentSource(space=uniform_dist, energy=energy_dist)

In [123]:
batches = 500
inactive = 20
particles = 10000

# Instantiate a Settings object
settings = openmc.Settings()

settings.source = source
settings.batches = batches
settings.inactive = inactive
settings.particles = particles
settings.run_mode = 'eigenvalue'

my_path = oralloy.name
settings.output = {'tallies': False, 'path':my_path}

model.settings = settings

In [124]:
groups = openmc.mgxs.EnergyGroups(group_edges='VITAMIN-J-42')
print(groups.group_edges)

[1.00e+03 1.00e+04 2.00e+04 3.00e+04 4.50e+04 6.00e+04 7.00e+04 7.50e+04
 1.00e+05 1.50e+05 2.00e+05 3.00e+05 4.00e+05 4.50e+05 5.10e+05 5.12e+05
 6.00e+05 7.00e+05 8.00e+05 1.00e+06 1.33e+06 1.34e+06 1.50e+06 1.66e+06
 2.00e+06 2.50e+06 3.00e+06 3.50e+06 4.00e+06 4.50e+06 5.00e+06 5.50e+06
 6.00e+06 6.50e+06 7.00e+06 7.50e+06 8.00e+06 1.00e+07 1.20e+07 1.40e+07
 2.00e+07 3.00e+07 5.00e+07]


In [125]:
lanl30_edges= [
    1.00e-5,  # Thermal cutoff (lower bound)
    1.00e+0, 5.00e+0, 1.00e+1, 5.00e+1, 1.00e+2, 3.00e+2, 5.00e+2, 1.00e+3, 2.00e+3,
    3.00e+3, 5.00e+3, 1.00e+4, 2.00e+4, 3.00e+4, 5.00e+4, 8.00e+4, 1.00e+5, 1.50e+5,
    2.00e+5, 3.00e+5, 4.00e+5, 6.00e+5, 8.00e+5, 1.00e+6, 1.50e+6, 2.00e+6, 3.00e+6,
    5.00e+6, 1.00e+7, 2.00e+7  # Upper bound (fast neutrons)
]
groups = openmc.mgxs.EnergyGroups(group_edges=lanl30_edges)
print(groups.group_edges)

[1.0e-05 1.0e+00 5.0e+00 1.0e+01 5.0e+01 1.0e+02 3.0e+02 5.0e+02 1.0e+03
 2.0e+03 3.0e+03 5.0e+03 1.0e+04 2.0e+04 3.0e+04 5.0e+04 8.0e+04 1.0e+05
 1.5e+05 2.0e+05 3.0e+05 4.0e+05 6.0e+05 8.0e+05 1.0e+06 1.5e+06 2.0e+06
 3.0e+06 5.0e+06 1.0e+07 2.0e+07]


In [107]:
# Initialize MGXS Library
mgxs_lib = openmc.mgxs.Library(model.geometry)

# Add the chosen group structure
mgxs_lib.energy_groups = groups

# Scatttering Format and Legendre Order
mgxs_lib.scatter_format = "legendre"
mgxs_lib.legendre_order = 3

# Specify multi-group cross-section types to compute
mgxs_lib.mgxs_types = ['total', 'absorption', 'scatter', 'fission', 'nu-fission', 'chi', 'reduced absorption', 'scatter matrix', 'nu-scatter matrix',\
                       'consistent nu-scatter matrix', 'multiplicity matrix']

# set uncertainty goal
mgxs_lib.tally_trigger = openmc.Trigger('std_dev', 1e-4)

# Compute cross sections on a nuclide-by-nuclide basis
mgxs_lib.by_nuclide = False

# Specify a "cell" domain type for the cross section tally filters
mgxs_lib.domain_type = 'material'

# Specify the cell domains over which to compute multi-group cross sections
mgxs_lib.domains = model.materials

# Construct all tallies needed for the multi-group cross section library
mgxs_lib.build_library()

tallies = openmc.Tallies()
mgxs_lib.add_to_tallies_file(tallies, merge=True)
model.tallies = tallies

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mgxs/library.py:401: RuntimeWarning: The P0 correction will be ignored since the scattering order 3 is greater than zero
  warn(msg, RuntimeWarning)


In [108]:
 model.export_to_model_xml(path = my_path + '/' + mat_cell.name + '.xml')

/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=104.
  warn(msg, IDWarning)
/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=105.
  warn(msg, IDWarning)
/home/jae123p/miniconda3/envs/openmc-env/lib/python3.12/site-packages/openmc/mixin.py:70: IDWarning: Another Filter instance already exists with id=115.
  warn(msg, IDWarning)


In [109]:
sp_file = model.run()

[LAPTOP-H8MU6HJU:04709] shmem: mmap: an error occurred while determining whether or not /tmp/ompi.LAPTOP-H8MU6HJU.1000/jf.0/2859663360/shared_mem_cuda_pool.LAPTOP-H8MU6HJU could be created.
[LAPTOP-H8MU6HJU:04709] create_and_attach: unable to create shared memory BTL coordinating structure :: size 134217728 
                                %%%%%%%%%%%%%%%
                           %%%%%%%%%%%%%%%%%%%%%%%%
                        %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                      %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                    %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                   %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
                                    %%%%%%%%%%%%%%%%%%%%%%%%
                                     %%%%%%%%%%%%%%%%%%%%%%%%
                 ###############      %%%%%%%%%%%%%%%%%%%%%%%%
                ##################     %%%%%%%%%%%%%%%%%%%%%%%
                ###################     %%%%%%%%%%%%%%%%%%%%%%%
                ####################     %%%%%

In [110]:
# Load the last statepoint file
sp = openmc.StatePoint(sp_file)

In [111]:
# Initialize MGXS Library with OpenMC statepoint data
mgxs_lib.load_from_statepoint(sp)

In [112]:
mgxs_file = mgxs_lib.create_mg_library(xs_type = 'macro')
mgxs_file.export_to_hdf5(filename = 'HEU-MET-Fast-003-1/oralloy.h5')

In [113]:
sp.close()